<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/model_experiments/model_diagnostics/model_diagnostics_guide.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/model_experiments/model_diagnostics/model_diagnostics_guide.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>


# Model Diagnostics Guide


Throughout the process of training your machine learning (ML) model, you may want to investigate your model's failures in order to understand which areas need improvement. Looking at an error analysis after each training iteration can help you understand whether you need to revise your annotations, make your ontology more clear, or create more training data that targets a specific area.
Labelbox now offers a Model Diagnostics tool that analyzes the performance of your model's predictions in a single interface.
With Model Diagnostics, you can:
*   Inspect model behavior across experiments
*   Adjust model hyperparameters and visualize model failures
*   Use the Python SDK to create the analysis pipeline

## How it works

Configuring Model Diagnostics is all done via the SDK. We have created a Google colab notebook to demonstrate this process. The notebook also includes a section that leverages MAL in order to quickly create ground truth annotations.
An Experiment is a specific instance of a model generating output in the form of predictions.
In Labelbox, the `Model` object represents your ML model and it is what you'll be performing experiments on. It references a set of annotations specified by an ontology. 
The `Model Run` object represents the experiment itself. It is a specific instance of a `Model` with preconfigured hyperparameters (training data). You can upload inferences across each `Model Run`, filter by IoU score, and compare your model's predictions against the annotations from your training data.

## Not ready to try with your model

For an end-to-end example with an existing dataset check out this [notebook](model_diagnostics_demo.ipynb)




# Steps
1. Make sure you are signed up for the beta. If not navigate here https://labelbox.com/product/model-diagnostics
2. Select a project
3. Exports labels
4. Upload labels and predictions for Diagnostics
------ 

## Environment Setup
* Install dependencies

In [ ]:
!pip install "labelbox[data]"

In [ ]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())

* Import libraries

In [ ]:
import uuid

from tqdm import notebook

import labelbox as lb
import labelbox.types as lb_types
from labelbox.data.metrics import feature_miou_metric, feature_confusion_matrix_metric
from labelbox.data.metrics.group import get_label_pairs

* Configure client

API Key

Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Select a project

In [ ]:
project_id = None

In [ ]:
project = client.get_project(project_id)
ontology = project.ontology()

## Export Labels

In [ ]:
MAX_LABELS = 2000
# we have a limit of 2000 labels
labels = [
    l for idx, l in enumerate(project.label_generator()) if idx < MAX_LABELS
]

## Create Predictions
* Loop over data_rows, make predictions, and annotation types

In [ ]:
predictions = list()
for label in notebook.tqdm(labels):
    annotations = []
    image = label.data

    ### --- replace (start) --- ###

    # Build a list of annotation objects from your model inferences
    # image.value is just the numpy array representation of the image
    prediction = model.predict(image.value)
    # Iterate of segmentation channels or instances depending on your model architecture ( or both )
    for instance, (xmin, ymin, xmax, ymax), seg, class_idx in prediction:
        class_name = class_names.get(class_idx)

        # Construct the right annotation value (pick one of the following) and append to list of annotations
        # See annotation types notebooks for more on how to construct these objects
        # https://github.com/Labelbox/labelbox-python/tree/master/examples/annotation_types

        value = lb_types.Polygon(points=[lb_types.Point(x=x, y=y) for x, y in instance])
        value = lb_types.Rectangle(start=lb_types.Point(x=xmin, y=ymin),
                          end=lb_types.Point(x=xmax, y=ymax))
        value = lb_types.Point(x=x, y=y)
        value = lb_types.Line(points=[lb_types.Point(x=x, y=y) for x, y in instance])
        value = lb_types.Mask(mask=lb_types.MaskData.from_2D_arr(seg * grayscale_color),
                     color=(grayscale_color,) * 3)

        annotations.append(lb_types.ObjectAnnotation(name=class_name, value=value))

    ### --- replace (end) --- ###
    predictions.append(lb_types.Label(data=image, annotations=annotations))

## Prepare for upload
* Add any mising urls or references to labelbox (data row ids)

In [ ]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
predictions.add_url_to_masks(signer)

## MEA

Throughout the process of training your machine learning (ML) model, you may want to investigate your model’s failures in order to understand which areas need improvement. Looking at an error analysis after each training iteration can help you understand whether you need to revise your annotations, make your ontology more clear, or create more training data that targets a specific area.





1. Create a model
    * Think of this as a model that you want to perform experiments on
2. Create a model run
    * Think of this as a single experiment for a particular model.
    * E.g. this model run is for an instance of a model with particular hyperparameters
3. Select the ground truth annotations for analysis
4. Compute metrics
4. Upload model predictions and metrics to labelbox

In [ ]:
lb_model = client.create_model(name=f"{project.name}-model",
                               ontology_id=project.ontology().uid)
lb_model_run_hyperparameters = {"batch_size": 1000}
lb_model_run = lb_model.create_model_run("0.0.0", lb_model_run_hyperparameters)
lb_model_run.upsert_labels([label.uid for label in labels])

In [ ]:
pairs = get_label_pairs(labels, predictions, filter_mismatch=True)
for (ground_truth, prediction) in pairs.values():
    metrics = []
    metrics.extend(
        feature_miou_metric(ground_truth.annotations, prediction.annotations))
    metrics.extend(
        feature_confusion_matrix_metric(ground_truth.annotations,
                                        prediction.annotations))
    prediction.annotations.extend(metrics)

In [ ]:
upload_task = lb_model_run.add_predictions(
    f'mea-import-{uuid.uuid4()}', predictions)

In [ ]:
upload_task.wait_until_done()
upload_task.state

### Open Model Run

In [ ]:
for idx, model_run_data_row in enumerate(lb_model_run.model_run_data_rows()):
    if idx == 5:
        break
    print(model_run_data_row.url)